In [3]:
!pip install numpy
!pip install tensorflow
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

     |████████████████████████████████| 123 kB 15.7 MB/s 


In [4]:
#adj, features, labels, train_mask, val_mask, test_mask 

cora= spektral.datasets.citation.Citation(name='cora', dtype='float32') 
graph = cora[0]

train_mask, val_mask, test_mask = cora.mask_tr, cora.mask_va, cora.mask_te
features, adj, labels = graph.x, graph.a, graph.y
#features = features.todense()
adj = adj + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))


(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [5]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)  # Computes softmax cross entropy between logits and labels
  mask = tf.cast(mask, dtype=tf.float32)   # Casts a tensor to a new type
  mask /= tf.reduce_mean(mask)       # reduce_mean: Computes the mean of elements across dimensions of a tensor.
  loss *= mask
  return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))  #  Returns the truth value of (x == y) element-wise
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask 
  return tf.reduce_mean(accuracy_all) 

In [6]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)   # Multiplies matrix a by matrix b, producing a * b
  return activation(ret_fts)

In [7]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)  # a regular densely-connected NN layer
  lyr_2 = tf.keras.layers.Dense(7)

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.nn.relu)
    return logits
  
  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  for ep in range(epochs + 1):
    with tf.GradientTape() as t:  # Record operations for automatic differentiation
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

      variables = t.watched_variables()
      grads = t.gradient(loss, variables)
      optimizer.apply_gradients(zip(grads, variables))

      logits = cora_gnn(fts, adj)
      val_accuracy = masked_accuracy(logits, labels, val_mask)
      test_accuracy = masked_accuracy(logits, labels, test_mask)

      if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:',
              val_accuracy.numpy(), '| Test accuracy:', test_accuracy.numpy())

In [36]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 4.3847504 | Val accuracy: 0.258 | Test accuracy: 0.26999998
Epoch 1 | Training loss: 2.419186 | Val accuracy: 0.42799997 | Test accuracy: 0.44299996
Epoch 2 | Training loss: 1.5790448 | Val accuracy: 0.52199996 | Test accuracy: 0.509
Epoch 4 | Training loss: 1.480004 | Val accuracy: 0.58599997 | Test accuracy: 0.602
Epoch 5 | Training loss: 0.4735535 | Val accuracy: 0.634 | Test accuracy: 0.63
Epoch 6 | Training loss: 0.43012133 | Val accuracy: 0.66800004 | Test accuracy: 0.681
Epoch 7 | Training loss: 0.44025835 | Val accuracy: 0.67599994 | Test accuracy: 0.714
Epoch 8 | Training loss: 0.26851663 | Val accuracy: 0.698 | Test accuracy: 0.726
Epoch 9 | Training loss: 0.20283346 | Val accuracy: 0.71199995 | Test accuracy: 0.73
Epoch 10 | Training loss: 0.15673612 | Val accuracy: 0.722 | Test accuracy: 0.737


In [9]:
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 1.9511968 | Val accuracy: 0.144 | Test accuracy: 0.14999999
Epoch 1 | Training loss: 1.8092537 | Val accuracy: 0.16799998 | Test accuracy: 0.18899998
Epoch 2 | Training loss: 1.6694698 | Val accuracy: 0.18399999 | Test accuracy: 0.20899998
Epoch 3 | Training loss: 1.5264202 | Val accuracy: 0.20999998 | Test accuracy: 0.22699998
Epoch 4 | Training loss: 1.3784618 | Val accuracy: 0.23199996 | Test accuracy: 0.265
Epoch 5 | Training loss: 1.2359295 | Val accuracy: 0.26399997 | Test accuracy: 0.294
Epoch 6 | Training loss: 1.1021346 | Val accuracy: 0.28599995 | Test accuracy: 0.31999996
Epoch 7 | Training loss: 0.9778144 | Val accuracy: 0.324 | Test accuracy: 0.342
Epoch 8 | Training loss: 0.86438 | Val accuracy: 0.35399997 | Test accuracy: 0.36699995
Epoch 9 | Training loss: 0.76459 | Val accuracy: 0.376 | Test accuracy: 0.38899994
Epoch 10 | Training loss: 0.6762422 | Val accuracy: 0.39999998 | Test accuracy: 0.408
Epoch 11 | Training loss: 0.5975513 | Val accura